# pattern for annex in table format

In [62]:
#table annex with numbers
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

#url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32011R1002'
url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32022D1907'
response = requests.get(url)
print(response.status_code)
data = response.text
#Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')
soup

200


<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml"><!-- CONVEX # converter_version:9.15.0 # generated_on:20240314-1128 # ELI version:0.10 --><head><meta content="telephone=no" name="format-detection"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="EN" name="WT.z_usr_lan"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type">
<script data-dtconfig="app=47d4c64c3b67ec69|agentId=1a806ce9da0a028c|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1722593265058|tp=500,50,0|agentUri=/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/eurlex-frontoffice/rb_39a3e95b-5423-482c-879b-99ef235dffeb|rid=RID_2066781245|rpid=2093193819|domain=europa.eu" src="/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_102892403251

In [68]:
test = soup.find_all('link',href=True)
#test = test.get_text(strip=False)
test[1]

<link href="https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022D1907" rel="canonical"/>

In [8]:
#get the document information

url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data) #finds the url through the href tag
#publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
#title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
title = soup.find('p', class_='oj-doc-ti')
title = test.get_text(strip=True)
celex= re.findall(r'uri=CELEX%([^"]+)', data) # get the celex id
publication_date = soup.find('p', class_='oj-hd-date')
publication_date = test.get_text(strip=True)


In [12]:
# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')#finda all elements tr = tablerow of the class oj-table

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td') #finds all td table data in one tablerow
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=False)  # Extracting the identifying information but keeping the newline 
                                                    #characters to later split by them
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)
    
    sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :title[0],
        'Publication Date': publication_date,
        'legal_base' : title[2],
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing
    })

# Display the results
for person in sanction_list:
    print(person)


{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022D1907'], 'CELEX': [], 'law_number': '\n            COUNCIL DECISION (CFSP) 2022/1907', 'Publication Date': ['6.10.2022'], 'legal_base': 'amending Decision 2014/145/CFSP concerning restrictive measures in respect of actions undermining or threatening the territorial integrity, sovereignty and independence of Ukraine', 'Name': 'Aleksander Dmitrievich KHARICHEV(Александр Дмитриевич ХАРИЧЕВ)', 'Identifying Information': '\nDOB: 8.2.1966\nPOB: Kostroma, former USSR (now Russian Federation)\nNationality: Russian\nGender: male\n', 'Reason': 'Head of the Division of Operation of the State Council in the Administration of the President of the Russian Federation. The Division has been tasked with the preparations for the illegal referenda in the occupied regions of Ukraine. He is therefore responsible for, supporting and implementing actions and policies which undermine or threaten the territorial integrity, sovereig

In [4]:

# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Extract additional document details using regex with checks
url_match = re.search(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"',data)
url = url_match.group(1) if url_match else None

publication_date_match = re.search(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
publication_date = publication_date_match.group(1) if publication_date_match else None

date_match = re.search(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>',data)
date = date_match.group(1) if date_match else None

title_match = re.search(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>',data)
title = title_match.group(1) if title_match else None

law_number_match = re.search(r'<p class="oj-hd-oj">C (\d+/\d+)</p>',data)
law_number = law_number_match.group(1) if law_number_match else None

case_number_match = re.search(r'Case\sT-\d+/\d+', data)
case_number = case_number_match.group(0) if case_number_match else None

legal_base_match = re.search(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
legal_base = legal_base_match.group(0) if legal_base_match else None

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=True)  # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)

    sanction_list.append({
        'URL': url,
        'Publication Date': publication_date,
        'Date': date,
        'Title': title,
        'Law Number': law_number,
        'Case Number': case_number,
        'Legal Base': legal_base,
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing
    })

# Display the results
import json
print(json.dumps(sanction_list, indent=4))


[
    {
        "URL": "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32022D1907",
        "Publication Date": "6.10.2022",
        "Date": null,
        "Title": "ANNEX",
        "Law Number": null,
        "Case Number": null,
        "Legal Base": "(CFSP) 2022/1907",
        "Name": "Aleksander Dmitrievich KHARICHEV(\u0410\u043b\u0435\u043a\u0441\u0430\u043d\u0434\u0440 \u0414\u043c\u0438\u0442\u0440\u0438\u0435\u0432\u0438\u0447 \u0425\u0410\u0420\u0418\u0427\u0415\u0412)",
        "Identifying Information": "DOB: 8.2.1966POB: Kostroma, former USSR (now Russian Federation)Nationality: RussianGender: male",
        "Reason": "Head of the Division of Operation of the State Council in the Administration of the President of the Russian Federation. The Division has been tasked with the preparations for the illegal referenda in the occupied regions of Ukraine. He is therefore responsible for, supporting and implementing actions and policies which undermine or threaten 